<a href="https://colab.research.google.com/github/toktok010/F550/blob/main/Text_Summarization_%26_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Text Summarization
!pip install transformers
from transformers import pipeline
summarizer = pipeline ("summarization", model="facebook/bart-large-cnn", truncation=True) ##set to True to truncate sequence to max permitted by the model
import requests
from bs4 import BeautifulSoup
''
url = "https://en.wikipedia.org/wiki/One_Hundred_and_One_Dalmatians"
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup (html_page, 'html.parser')
text = ''
for paragraph in soup.find_all('p'):
  text += paragraph.text
#import package
import re
#clean text
text = re.sub(r'\[.*?\]+', '', text)
text = text.replace('\n','')
text
summary=summarizer(text, max_length=250, min_length=30, do_sample=False)
print(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'summary_text': "One Hundred and One Dalmatians is a 1961 American animated adventure comedy film. Based on Dodie Smith's 1956 novel, it was the first animated feature to earn over $10 million during its initial release. The film's plot follows a litter of fifteen Dalmatian puppies, who are kidnapped by the obsessive socialite Cruella De Vil. Their parents, Pongo and Perdita, set out on a cross-country rescue mission."}]


In [ ]:
##Topic Modelling
!pip install bertopic
from bertopic import BERTopic
#data preprocessing
import pandas as pd
import numpy as np
#text preprocessing
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
#topic model
from bertopic import BERTopic
#dimension reduction
from umap import UMAP

#mount google drive
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 14.6 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.2 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039291 sha256=4841e72bd8098232906ba163d757fce94f075ec1af1b075399fe817cc4c59b83
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for umap-l

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


In [ ]:
#change directory
import os
os.chdir("/content/drive/MyDrive/NLP")
#print out the current directory
!pwd

/content/drive/MyDrive/NLP


In [ ]:
#read in data
amz_review = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])
#drop the label
amz_review = amz_review.drop('label', axis=1);
#take a look at the data
amz_review.head()

,review
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In [ ]:
#get the dataset info
amz_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [ ]:
#remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
print(f'There are {len(stopwords)} default stopwords. They are {stopwords}')
#Remove stopwords
amz_review['review_without_stopwords'] = amz_review['review'].apply(lambda x: ' '.join([w for w in x.split() if w.lower() not in stopwords]))
#Lemmatization
amz_review['review_lemmatized'] = amz_review['review_without_stopwords'].apply(lambda x: ' '.join([wn.lemmatize(w) for w in x.split() if w not in stopwords]))
#see the data
amz_review.head()

There are 179 default stopwords. They are ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

,review,review_without_stopwords,review_lemmatized
0,So there is no way for me to plug it in here i...,way plug US unless go converter.,way plug US unless go converter.
1,"Good case, Excellent value.","Good case, Excellent value.","Good case, Excellent value."
2,Great for the jawbone.,Great jawbone.,Great jawbone.
3,Tied to charger for conversations lasting more...,Tied charger conversations lasting 45 minutes....,Tied charger conversation lasting 45 minutes.M...
4,The mic is great.,mic great.,mic great.


In [ ]:
#initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)
#initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)
#run BERTopic model
topics, probabilities = topic_model.fit_transform(amz_review['review_lemmatized'])

In [ ]:
#get the list of topics
topic_model.get_topic_info()
#get top 10 terms for a topic
topic_model.get_topic(0)

[('sound', 0.12384355266363802),
 ('quality', 0.08070559890751694),
 ('volume', 0.061103883685725204),
 ('audio', 0.054300801120649146),
 ('poor', 0.04946183333947867),
 ('low', 0.041935570767540266),
 ('clear', 0.03818992730357825),
 ('hear', 0.03818992730357825),
 ('loud', 0.03784007938826608),
 ('end', 0.036200534080432764)]

In [ ]:
#visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=12)

In [ ]:
#visualize term rank decrease
topic_model.visualize_term_rank()

In [ ]:
#visualize intertopic distance
topic_model.visualize_topics()

In [ ]:
#visualize connections between topics using hierarchical clustering
topic_model.visualize_hierarchy(top_n_topics=10)

In [ ]:
#visualize similarity using heatmap
topic_model.visualize_heatmap()

In [ ]:
#visualize probability distribution
topic_model.visualize_distribution(topic_model.probabilities_[0], min_probability=0.015)